In [1]:
import statsmodels.api as sm
import statsmodels.stats.sandwich_covariance as sw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import data

In [2]:
OK = pd.read_excel(
    "data/OK.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

OK["Date"] = pd.to_datetime(OK["Date"])
OK = OK.set_index("Date").sort_index()

OK.head()

,Close
Date,
2020-01-02,54.99
2020-01-03,54.69
2020-01-06,54.67
2020-01-07,54.25
2020-01-08,54.35


In [3]:
SPY = pd.read_excel(
    "data/SPY.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

SPY["Date"] = pd.to_datetime(SPY["Date"])
SPY = SPY.set_index("Date").sort_index()

SPY.head()

,Close
Date,
2020-01-02,324.87
2020-01-03,322.41
2020-01-06,323.64
2020-01-07,322.73
2020-01-08,324.45


In [4]:
GCF = pd.read_excel(
    "data/GCF.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

GCF["Date"] = pd.to_datetime(GCF["Date"])
GCF = GCF.set_index("Date").sort_index()

GCF.head()

,Close
Date,
2020-01-02,1524.5
2020-01-03,1549.2
2020-01-06,1566.2
2020-01-07,1571.8
2020-01-08,1557.4


In [5]:
CRSP = pd.read_excel(
    "data/CRSP.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

CRSP["Date"] = pd.to_datetime(CRSP["Date"])
CRSP = CRSP.set_index("Date").sort_index()

CRSP.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
2020-01-02,0.86,-0.88,-0.34,0.006
2020-01-03,-0.67,0.38,0.01,0.006
2020-01-06,0.36,-0.07,-0.55,0.006
2020-01-07,-0.19,-0.01,-0.26,0.006
2020-01-08,0.47,-0.07,-0.64,0.006


## Calculate daily rate of return

In [6]:
# r(t+1) = ln[P(t+1) / P(t)]
OK["log_return"] = np.log(OK["Close"] / OK["Close"].shift(1))

# r(t+1) = [P(t+1) - P(t)] / P(t)
OK["arith_return"] = OK["Close"].pct_change()

OK.head()

,Close,log_return,arith_return
Date,,,
2020-01-02,54.99,NaN,NaN
2020-01-03,54.69,-0.005470,-0.005456
2020-01-06,54.67,-0.000366,-0.000366
2020-01-07,54.25,-0.007712,-0.007682
2020-01-08,54.35,0.001842,0.001843


In [7]:
# r(t+1) = ln[P(t+1) / P(t)]
SPY["log_return"] = np.log(SPY["Close"] / SPY["Close"].shift(1))

# r(t+1) = [P(t+1) - P(t)] / P(t)
SPY["arith_return"] = SPY["Close"].pct_change()

SPY.head()

,Close,log_return,arith_return
Date,,,
2020-01-02,324.87,NaN,NaN
2020-01-03,322.41,-0.007601,-0.007572
2020-01-06,323.64,0.003808,0.003815
2020-01-07,322.73,-0.002816,-0.002812
2020-01-08,324.45,0.005315,0.005330


In [8]:
# r(t+1) = ln[P(t+1) / P(t)]
GCF["log_return"] = np.log(GCF["Close"] / GCF["Close"].shift(1))

# r(t+1) = [P(t+1) - P(t)] / P(t)
GCF["arith_return"] = GCF["Close"].pct_change()

GCF.head()

,Close,log_return,arith_return
Date,,,
2020-01-02,1524.5,NaN,NaN
2020-01-03,1549.2,0.016072,0.016202
2020-01-06,1566.2,0.010914,0.010973
2020-01-07,1571.8,0.003569,0.003576
2020-01-08,1557.4,-0.009204,-0.009161


## Stock data

In [9]:
# 选用对数收益率列并去掉 NaN
KO_ret = OK["log_return"].dropna()

# 计算统计量
stats = {
    "Name"       : "Coca-Cola",
    "Ticker"     : "KO",
    "Mean"       : KO_ret.mean(),
    "Median"     : KO_ret.median(),
    "Std Dev"    : KO_ret.std(ddof=1),   # 样本标准差
    "Min"        : KO_ret.min(),
    "Max"        : KO_ret.max(),
    "Skewness"   : KO_ret.skew(),
    "Kurtosis"   : KO_ret.kurt(),
    "Start Date" : KO_ret.index.min().date(),
    "End Date"   : KO_ret.index.max().date()
}


summary = pd.DataFrame(stats, index=["Value"]).T
print(summary.to_string())

                 Value
Name         Coca-Cola
Ticker              KO
Mean          0.000193
Median        0.000958
Std Dev       0.015624
Min          -0.101728
Max           0.062783
Skewness     -0.841427
Kurtosis      6.966093
Start Date  2020-01-03
End Date    2022-12-30


In [10]:
# 选用对数收益率列并去掉 NaN
SPY_ret = SPY["log_return"].dropna()

# 计算统计量
stats = {
    "Name"       : "SPDR S&P 500 ETF",
    "Ticker"     : "SPY",
    "Mean"       : SPY_ret.mean(),
    "Median"     : SPY_ret.median(),
    "Std Dev"    : SPY_ret.std(ddof=1),   # 样本标准差
    "Min"        : SPY_ret.min(),
    "Max"        : SPY_ret.max(),
    "Skewness"   : SPY_ret.skew(),
    "Kurtosis"   : SPY_ret.kurt(),
    "Start Date" : SPY_ret.index.min().date(),
    "End Date"   : SPY_ret.index.max().date()
}

summary = pd.DataFrame(stats, index=["Value"]).T
print(summary.to_string())

                       Value
Name        SPDR S&P 500 ETF
Ticker                   SPY
Mean                0.000216
Median              0.000853
Std Dev             0.015876
Min                -0.115887
Max                 0.086731
Skewness           -0.736754
Kurtosis            9.408741
Start Date        2020-01-03
End Date          2022-12-30


## Separate the data

In [11]:
# --- 函数：计算描述性统计 -------------------------------------------------
def get_stats(series):
    return {
        "Mean"     : series.mean(),
        "Median"   : series.median(),
        "Std Dev"  : series.std(ddof=1),
        "Min"      : series.min(),
        "Max"      : series.max(),
        "Skewness" : series.skew(),
        "Kurtosis" : series.kurt()
    }

# --- 1. 按时间切片 --------------------------------------------------------
train_OK = OK.loc["2020":"2021", "log_return"].dropna()   # 前两年
test_OK  = OK.loc["2022", "log_return"].dropna() # 最后一年

# --- 2. 生成统计表 ---------------------------------------------------------
stats_train_OK = pd.DataFrame(get_stats(train_OK), index=["2020-2021"]).T
stats_test_OK  = pd.DataFrame(get_stats(test_OK),  index=["2022"]).T

print("\n===== KO-train (2020-2021) =====")
print(stats_train_OK.to_string())

print("\n===== KO-test (2022) =====")
print(stats_test_OK.to_string())


===== KO-train (2020-2021) =====
          2020-2021
Mean       0.000147
Median     0.000751
Std Dev    0.016984
Min       -0.101728
Max        0.062783
Skewness  -0.815080
Kurtosis   6.624366

===== KO-test (2022) =====
              2022
Mean      0.000286
Median    0.001399
Std Dev   0.012482
Min      -0.072169
Max       0.037942
Skewness -0.835994
Kurtosis  4.678076


In [12]:
# --- 1. 按时间切片 --------------------------------------------------------
train_SPY = SPY.loc["2020":"2021", "log_return"].dropna()   # 前两年
test_SPY  = SPY.loc["2022", "log_return"].dropna() # 最后一年

# --- 2. 生成统计表 ---------------------------------------------------------
stats_train_SPY = pd.DataFrame(get_stats(train_SPY), index=["2020-2021"]).T
stats_test_SPY  = pd.DataFrame(get_stats(test_SPY),  index=["2022"]).T

print("\n===== SPY-train (2020-2021) =====")
print(stats_train_SPY.to_string())

print("\n===== SPY-test (2022) =====")
print(stats_test_SPY.to_string())


===== SPY-train (2020-2021) =====
          2020-2021
Mean       0.000754
Median     0.001648
Std Dev    0.016144
Min       -0.115887
Max        0.086731
Skewness  -1.061486
Kurtosis  13.253851

===== SPY-test (2022) =====
              2022
Mean     -0.000863
Median   -0.001815
Std Dev   0.015298
Min      -0.044456
Max       0.053497
Skewness -0.001245
Kurtosis  0.332860


In [13]:
# --- 按时间切片 --------------------------------------------------------
train_GCF = GCF.loc["2020":"2021", "log_return"].dropna()   # 前两年
test_GCF  = GCF.loc["2022", "log_return"].dropna() # 最后一年


CRSP["Mkt_Total"] = CRSP["Mkt-RF"] + CRSP["RF"]
train_CRSP = CRSP.loc["2020":"2021", "Mkt_Total"].dropna()   # 前两年
test_CRSP  = CRSP.loc["2022", "Mkt_Total"].dropna()          # 最后一年

## OLS & Newey-West

### OK

In [14]:
# X: intercept + time trend based on the DatetimeIndex                            
t_train = train_OK.index.map(pd.Timestamp.toordinal).to_numpy() # convert dates → integers
X_train = sm.add_constant(t_train) # add intercept

y_train = train_OK.values

# OLS
model = sm.OLS(y_train, X_train).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.7580
Date:                Wed, 18 Jun 2025   Prob (F-statistic):              0.384
Time:                        21:10:47   Log-Likelihood:                 1339.8
No. Observations:                 504   AIC:                            -2676.
Df Residuals:                     502   BIC:                            -2667.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3085      2.652     -0.871      0.3

In [15]:
hac_cov = sw.cov_hac(model, 1)

s = model.get_robustcov_results(cov_type='HAC', maxlags=1)
print(s.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5620
Date:                Wed, 18 Jun 2025   Prob (F-statistic):              0.454
Time:                        21:10:47   Log-Likelihood:                 1339.8
No. Observations:                 504   AIC:                            -2676.
Df Residuals:                     502   BIC:                            -2667.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3085      3.080     -0.750      0.4

## 4 estimated regressions

### (1) Intercept, benchmark returns

In [16]:
train1 = pd.concat([
    train_OK.rename("KO_logRet"),
    train_SPY.rename("SPY_logRet")
], axis=1)

y1 = train1["KO_logRet"]
X1 = train1["SPY_logRet"]

X1c = (sm.add_constant(X1, has_constant='add')
          .rename(columns={"const": "Intercept"}))

# OLS
ols1 = sm.OLS(y1, X1c).fit()
print("=== OLS ===")
print(ols1.summary())

=== OLS ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     669.3
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           2.05e-94
Time:                        21:10:47   Log-Likelihood:                 1552.9
No. Observations:                 504   AIC:                            -3102.
Df Residuals:                     502   BIC:                            -3093.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0005      0.000     -0.

In [17]:
# Newey-West (lag = 1)
hac1 = ols1.get_robustcov_results(cov_type="HAC", maxlags=1)
print("\n=== Newey-West (lag = 1) ===")
print(hac1.summary())


=== Newey-West (lag = 1) ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.571
Method:                 Least Squares   F-statistic:                     250.8
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           4.15e-46
Time:                        21:10:47   Log-Likelihood:                 1552.9
No. Observations:                 504   AIC:                            -3102.
Df Residuals:                     502   BIC:                            -3093.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0005 

### (2) Intercept, CRSP returns

In [18]:
train2 = pd.concat([
    train_OK.rename("KO_logRet"),
    train_CRSP.rename("CRSP_logRet")
], axis=1).dropna()

y2  = train2["KO_logRet"]
X2  = train2["CRSP_logRet"]
X2c = (sm.add_constant(X2, has_constant='add')
          .rename(columns={"const": "Intercept"}))

ols2 = sm.OLS(y2, X2c).fit()
print("\n=== OLS ===")
print(ols2.summary())


=== OLS ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     538.7
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           1.67e-81
Time:                        21:10:48   Log-Likelihood:                 1523.1
No. Observations:                 504   AIC:                            -3042.
Df Residuals:                     502   BIC:                            -3034.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0006      0.001    

In [19]:
# Newey-West (lag = 1)
hac2 = ols2.get_robustcov_results(cov_type="HAC", maxlags=1)
print("\n=== Newey-West (lag = 1) ===")
print(hac2.summary())


=== Newey-West (lag = 1) ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     227.0
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           1.35e-42
Time:                        21:10:48   Log-Likelihood:                 1523.1
No. Observations:                 504   AIC:                            -3042.
Df Residuals:                     502   BIC:                            -3034.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.00

### (3) Intercept, benchmark returns, X2 returns

In [20]:
train3 = pd.concat([
    train_OK.rename("KO_logRet"),
    train_SPY.rename("SPY_logRet"),
    train_GCF.rename("Gold_logRet")
], axis=1)

y3 = train3["KO_logRet"]
X3 = train3[["SPY_logRet", "Gold_logRet"]]

X3c = (sm.add_constant(X3, has_constant='add')
          .rename(columns={"const": "Intercept"}))

# OLS
ols3 = sm.OLS(y3, X3c).fit()
print("=== OLS ===")
print(ols3.summary())

=== OLS ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     334.3
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           5.85e-93
Time:                        21:10:48   Log-Likelihood:                 1553.0
No. Observations:                 504   AIC:                            -3100.
Df Residuals:                     501   BIC:                            -3087.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0005      0.000     

In [21]:
# Newey-West (lag = 1)
hac3 = ols3.get_robustcov_results(cov_type="HAC", maxlags=1)
print("\n=== Newey-West (lag = 1) ===")
print(hac3.summary())


=== Newey-West (lag = 1) ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.572
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     132.2
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           7.70e-47
Time:                        21:10:48   Log-Likelihood:                 1553.0
No. Observations:                 504   AIC:                            -3100.
Df Residuals:                     501   BIC:                            -3087.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.00

### (4) Intercept, CRSP returns, X2 returns

In [22]:
train4 = pd.concat([
    train_OK.rename("KO_logRet"),
    train_CRSP.rename("CRSP_logRet"),
    train_GCF.rename("Gold_logRet")
], axis=1).dropna()

y4  = train4["KO_logRet"]
X4  = train4[["CRSP_logRet", "Gold_logRet"]]
X4c = (sm.add_constant(X4, has_constant='add')
          .rename(columns={"const": "Intercept"}))

ols4 = sm.OLS(y4, X4c).fit()
print("\n=== OLS ===")
print(ols4.summary())


=== OLS ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     269.0
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           4.51e-80
Time:                        21:10:48   Log-Likelihood:                 1523.2
No. Observations:                 504   AIC:                            -3040.
Df Residuals:                     501   BIC:                            -3028.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0006      0.001    

In [23]:
hac4 = ols4.get_robustcov_results(cov_type="HAC", maxlags=1)
print("\n=== Newey-West (lag = 1) ===")
print(hac4.summary())


=== Newey-West (lag = 1) ===
                            OLS Regression Results                            
Dep. Variable:              KO_logRet   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     117.3
Date:                Wed, 18 Jun 2025   Prob (F-statistic):           1.60e-42
Time:                        21:10:48   Log-Likelihood:                 1523.2
No. Observations:                 504   AIC:                            -3040.
Df Residuals:                     501   BIC:                            -3028.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.00